In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
df2016 = pd.read_csv("nifty2016.csv")
df2017 = pd.read_csv("nifty2017.csv")
df2018 = pd.read_csv("nifty2018.csv")
df2019 = pd.read_csv("nifty2019.csv")
df2020 = pd.read_csv("nifty2020.csv")
df2021 = pd.read_csv("nifty2021.csv")
dfin = df2016.append(df2017)
dfin = dfin.append(df2018)
dfin = dfin.append(df2019)
dfin = dfin.append(df2020)
dfin = dfin.append(df2021)
# dfin.drop(['Shares Traded','Close','Open','Turnover (Rs. Cr)'],inplace=True,axis=1)
# dfin['Close'] = (dfin['Low'] + dfin['High']) / 2
# dfin.drop(['Low','High'],inplace=True,axis=1)
dfin['Gap'] = dfin['Open'] - dfin['Close'].shift(1)
dfin['Gap'] = dfin['Gap'].fillna(0)
dfin.drop(['Shares Traded','Low','Turnover (Rs. Cr)'],inplace=True,axis=1)
dfin.head()

,Date,Open,High,Close,Gap
0,26-Dec-2016,7965.10,7970.05,7908.25,0.00
1,27-Dec-2016,7915.05,8044.65,8032.85,6.80
2,28-Dec-2016,8047.55,8100.55,8034.85,14.70
3,29-Dec-2016,8030.60,8111.10,8103.60,-4.25
4,30-Dec-2016,8119.65,8197.00,8185.80,16.05


In [2]:
df = dfin.copy()
df['Date'] = df['Date'].apply(lambda x : datetime.strptime(x,"%d-%b-%Y"))
#df.head(20)

In [3]:
df['Day'] = df['Date'].apply(lambda x: x.weekday())
df['Month'] = df['Date'].apply(lambda x: x.month)
df['MonthYear'] = df['Date'].apply(lambda x: str(x.year) + str(x.month))
df.head()

,Date,Open,High,Close,Gap,Day,Month,MonthYear
0,2016-12-26,7965.10,7970.05,7908.25,0.00,0,12,201612
1,2016-12-27,7915.05,8044.65,8032.85,6.80,1,12,201612
2,2016-12-28,8047.55,8100.55,8034.85,14.70,2,12,201612
3,2016-12-29,8030.60,8111.10,8103.60,-4.25,3,12,201612
4,2016-12-30,8119.65,8197.00,8185.80,16.05,4,12,201612


In [4]:
def move_to_next_day(row):
    return_date = row['15dayback']
    if row['15dayback'].weekday() == 5:
        return_date = row['15dayback'] - timedelta(days=1)
    elif row['15dayback'].weekday() == 6:
        return_date = row['15dayback'] - timedelta(days=2)
    return return_date

def select_buy_date(row):
    dt = row['Date'].day // 2
    sel_date = row['Date'] - timedelta(days=dt)
    return sel_date

In [5]:
df_lastthursday = df[(df['Day'] == 3) | (df['Day'] == 2)].copy()
df_lastthursday = df_lastthursday.groupby(['MonthYear']).tail(1).copy().reset_index(drop=True)
# window = 15
df_lastthursday['15dayback'] = df_lastthursday.apply(select_buy_date, axis=1)
print(df_lastthursday.tail())
df_lastthursday['15dayback'] = df_lastthursday.apply(move_to_next_day, axis=1)
df_lastthursday['row_num'] = df_lastthursday.index
# df_lastday['15dayWeekday'] = df_lastday.apply(lambda x : x['15day'].weekday(),axis=1)
# df_lastday['row_num'] = df_lastday.index
df_lastthursday.tail()


         Date      Open      High     Close     Gap  Day  Month MonthYear  \
56 2021-08-26  16627.95  16683.70  16636.90   -6.70    3      8     20218   
57 2021-09-30  17718.90  17742.15  17618.15    7.60    3      9     20219   
58 2021-10-28  18187.65  18190.70  17857.25  -23.30    3     10    202110   
59 2021-11-25  17417.30  17564.35  17536.25    2.25    3     11    202111   
60 2021-12-16  17373.00  17379.35  17248.40  151.60    3     12    202112   

    15dayback  
56 2021-08-13  
57 2021-09-15  
58 2021-10-14  
59 2021-11-13  
60 2021-12-08  


,Date,Open,High,Close,Gap,Day,Month,MonthYear,15dayback,row_num
56,2021-08-26,16627.95,16683.70,16636.90,-6.70,3,8,20218,2021-08-13,56
57,2021-09-30,17718.90,17742.15,17618.15,7.60,3,9,20219,2021-09-15,57
58,2021-10-28,18187.65,18190.70,17857.25,-23.30,3,10,202110,2021-10-14,58
59,2021-11-25,17417.30,17564.35,17536.25,2.25,3,11,202111,2021-11-12,59
60,2021-12-16,17373.00,17379.35,17248.40,151.60,3,12,202112,2021-12-08,60


In [6]:
df_15day = df[['Date','Close']].copy()
df_mid = pd.merge(df_lastthursday, df_15day, left_on=["15dayback"], right_on=["Date"], how="left")
df_mid = df_mid[1::]
df_mid.rename(columns = {'Date_x':'Date_Buy', 'Close_x':'Close_Buy', 'Date_y':'Date_Sell', 'Close_y':'Close_Sell'}, inplace = True)

df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Sell'] == 0)].copy()
# df_err["15daynew"] = df_err["15day"] + timedelta(days=1)
print(len(df_err))
df_mid.head(25)


3


,Date_Buy,Open,High,Close_Buy,Gap,Day,Month,MonthYear,15dayback,row_num,Date_Sell,Close_Sell
1,2017-01-25,8499.45,8612.60,8602.75,23.65,2,1,20171,2017-01-13,1,2017-01-13 00:00:00,8400.35
2,2017-02-23,8956.40,8982.15,8939.50,29.50,3,2,20172,2017-02-10,2,2017-02-10 00:00:00,8793.55
3,2017-03-30,9142.60,9183.15,9173.75,-1.20,3,3,20173,2017-03-15,3,2017-03-15 00:00:00,9084.80
4,2017-04-27,9359.15,9367.15,9342.15,7.30,3,4,20174,2017-04-14,4,0,0.00
5,2017-05-31,9636.55,9649.60,9621.25,12.00,2,5,20175,2017-05-16,5,2017-05-16 00:00:00,9512.25
6,2017-06-29,9522.95,9575.80,9504.10,31.70,3,6,20176,2017-06-15,6,2017-06-15 00:00:00,9578.05
7,2017-07-27,10063.25,10114.85,10020.55,42.60,3,7,20177,2017-07-14,7,2017-07-14 00:00:00,9886.35
8,2017-08-31,9905.70,9925.10,9917.90,21.30,3,8,20178,2017-08-16,8,2017-08-16 00:00:00,9897.30
9,2017-09-28,9736.40,9789.20,9768.95,0.65,3,9,20179,2017-09-14,9,2017-09-14 00:00:00,10086.60
10,2017-10-26,10291.80,10355.65,10343.80,-3.55,3,10,201710,2017-10-13,10,2017-10-13 00:00:00,10167.45


In [7]:
# df_lastthursday.head(25)

In [8]:
for index, row in df_err.iterrows():
#     print(row['row_num'], row['15day'])
    new_date = df[(df['Date'] >= row['15dayback'])].head(1)['Date']
#     print(row['row_num'], row['15day'], new_date)
    if list(new_date):
        df_lastthursday.at[row['row_num'],'15dayback'] = list(new_date)[0]
#     else:
#         df_lastday = df_lastday.drop(index=row['row_num'])
df_lastthursday.head(25)

,Date,Open,High,Close,Gap,Day,Month,MonthYear,15dayback,row_num
0,2016-12-29,8030.60,8111.10,8103.60,-4.25,3,12,201612,2016-12-15,0
1,2017-01-25,8499.45,8612.60,8602.75,23.65,2,1,20171,2017-01-13,1
2,2017-02-23,8956.40,8982.15,8939.50,29.50,3,2,20172,2017-02-10,2
3,2017-03-30,9142.60,9183.15,9173.75,-1.20,3,3,20173,2017-03-15,3
4,2017-04-27,9359.15,9367.15,9342.15,7.30,3,4,20174,2017-04-17,4
5,2017-05-31,9636.55,9649.60,9621.25,12.00,2,5,20175,2017-05-16,5
6,2017-06-29,9522.95,9575.80,9504.10,31.70,3,6,20176,2017-06-15,6
7,2017-07-27,10063.25,10114.85,10020.55,42.60,3,7,20177,2017-07-14,7
8,2017-08-31,9905.70,9925.10,9917.90,21.30,3,8,20178,2017-08-16,8
9,2017-09-28,9736.40,9789.20,9768.95,0.65,3,9,20179,2017-09-14,9


In [9]:
df_mid = pd.merge(df_lastthursday, df_15day, left_on=["15dayback"], right_on=["Date"], how="left")
df_mid.rename(columns = {'Date_x':'Date_Buy', 'Close_x':'Close_Buy', 'Date_y':'Date_Sell', 'Close_y':'Close_Sell'}, inplace = True)
df_mid = df_mid[1::]
df_mid.fillna(0, inplace=True)
df_err = df_mid[(df_mid['Close_Sell'] == 0)].copy()
if len(df_err) > 0:
    print(df_err.tail(10))
    print("Error")
else: print("No Errors")
# df_mid['Date_Buy'] = df_mid.apply(lambda x: x['Date_Buy'].date(), axis=1)
df_mid.head(10)

No Errors


,Date_Buy,Open,High,Close_Buy,Gap,Day,Month,MonthYear,15dayback,row_num,Date_Sell,Close_Sell
1,2017-01-25,8499.45,8612.60,8602.75,23.65,2,1,20171,2017-01-13,1,2017-01-13,8400.35
2,2017-02-23,8956.40,8982.15,8939.50,29.50,3,2,20172,2017-02-10,2,2017-02-10,8793.55
3,2017-03-30,9142.60,9183.15,9173.75,-1.20,3,3,20173,2017-03-15,3,2017-03-15,9084.80
4,2017-04-27,9359.15,9367.15,9342.15,7.30,3,4,20174,2017-04-17,4,2017-04-17,9139.30
5,2017-05-31,9636.55,9649.60,9621.25,12.00,2,5,20175,2017-05-16,5,2017-05-16,9512.25
6,2017-06-29,9522.95,9575.80,9504.10,31.70,3,6,20176,2017-06-15,6,2017-06-15,9578.05
7,2017-07-27,10063.25,10114.85,10020.55,42.60,3,7,20177,2017-07-14,7,2017-07-14,9886.35
8,2017-08-31,9905.70,9925.10,9917.90,21.30,3,8,20178,2017-08-16,8,2017-08-16,9897.30
9,2017-09-28,9736.40,9789.20,9768.95,0.65,3,9,20179,2017-09-14,9,2017-09-14,10086.60
10,2017-10-26,10291.80,10355.65,10343.80,-3.55,3,10,201710,2017-10-13,10,2017-10-13,10167.45


In [10]:
df_mid.drop(["Day","Month","15dayback","row_num"],axis=1,inplace=True)
df_mid['Year'] = df_mid['Date_Sell'].apply(lambda x: x.year)
df_mid['BeginYear'] = df_mid['Year'].diff(periods=-1)
df_mid.loc[df_mid.index[-1], 'BeginYear']= -1
df_mid.head(25)

,Date_Buy,Open,High,Close_Buy,Gap,MonthYear,Date_Sell,Close_Sell,Year,BeginYear
1,2017-01-25,8499.45,8612.60,8602.75,23.65,20171,2017-01-13,8400.35,2017,0.0
2,2017-02-23,8956.40,8982.15,8939.50,29.50,20172,2017-02-10,8793.55,2017,0.0
3,2017-03-30,9142.60,9183.15,9173.75,-1.20,20173,2017-03-15,9084.80,2017,0.0
4,2017-04-27,9359.15,9367.15,9342.15,7.30,20174,2017-04-17,9139.30,2017,0.0
5,2017-05-31,9636.55,9649.60,9621.25,12.00,20175,2017-05-16,9512.25,2017,0.0
6,2017-06-29,9522.95,9575.80,9504.10,31.70,20176,2017-06-15,9578.05,2017,0.0
7,2017-07-27,10063.25,10114.85,10020.55,42.60,20177,2017-07-14,9886.35,2017,0.0
8,2017-08-31,9905.70,9925.10,9917.90,21.30,20178,2017-08-16,9897.30,2017,0.0
9,2017-09-28,9736.40,9789.20,9768.95,0.65,20179,2017-09-14,10086.60,2017,0.0
10,2017-10-26,10291.80,10355.65,10343.80,-3.55,201710,2017-10-13,10167.45,2017,0.0


In [11]:
#df_lastday.tail(20)
# spread, profit_positive, profit_negative = 0, 3300, 6700
# OTM, spread, profit_positive, profit_negative= 0, 200, 3250, 10000
# OTM, spread, profit_positive, profit_negative = 200, 100, 1650, 5000 
# OTM, spread, profit_positive, profit_negative = 100, 200, 1350, 10000 
# OTM, spread, profit_positive, profit_negative = 200, 200, 2550, 10000 
# OTM, profit_positive, profit_negative = 100, 6250, -7500
OTM, profit_positive, profit_negative = 300, 2500, -5000
# OTM, profit_positive, profit_negative = 200, 5000, -5000

In [12]:
dfsellput = df_mid.copy()
dfsellput['StrikePrice'] = dfsellput['Close_Sell'].apply(lambda x: round(x + OTM,-2))
dfsellput.head(5)

,Date_Buy,Open,High,Close_Buy,Gap,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,StrikePrice
1,2017-01-25,8499.45,8612.60,8602.75,23.65,20171,2017-01-13,8400.35,2017,0.0,8700.0
2,2017-02-23,8956.40,8982.15,8939.50,29.50,20172,2017-02-10,8793.55,2017,0.0,9100.0
3,2017-03-30,9142.60,9183.15,9173.75,-1.20,20173,2017-03-15,9084.80,2017,0.0,9400.0
4,2017-04-27,9359.15,9367.15,9342.15,7.30,20174,2017-04-17,9139.30,2017,0.0,9400.0
5,2017-05-31,9636.55,9649.60,9621.25,12.00,20175,2017-05-16,9512.25,2017,0.0,9800.0


In [13]:
# dfsellput.to_csv("nifty_dfsellput_OTM" + str(OTM) + "_Spread" + str(spread) + "_" + str(window) + "days.csv")

In [14]:
max_high = []
max_date = []
gap = []
gap_open = []
gap_date = []
for i, r in  dfsellput.iterrows():
#     print(r['Date_Sell'],r['Date_Buy'])
    dfw = df[(df["Date"] > r['Date_Sell']) & (df["Date"] <= r['Date_Buy'])][['Date','Close','High','Gap','Open']]
    dfw1 = dfw[(dfw['High'] >= r['StrikePrice'])]
    if len(dfw1):
        cl = dfw1['High'].min()
        max_high.append(cl)
        dt = dfw1[dfw1['High'] == cl]
        max_date.append(list(dt['Date'])[0])
    else:
        max_date.append(r['Date_Buy'])
        max_high.append(r['Close_Buy'])
    
    gcl = dfw['Gap'].max()
    
    gap.append(gcl)
    gdt = dfw[dfw['Gap'] == gcl]
    gap_date.append(list(gdt['Date'])[0])
    gap_open.append(list(gdt['Open'])[0])
    
dfsellput['Max_High'] = max_high
dfsellput['Max_Date'] = max_date
dfsellput['Gap'] = gap
dfsellput['Gap_Open'] = gap_open
dfsellput['Gap_Date'] = gap_date
dfsellput['Max_Days'] = (dfsellput['Max_Date'] - dfsellput['Date_Sell']).dt.days
dfsellput['Pull_Up_Strike'] = dfsellput.apply(lambda x: x['Max_High'] - x['StrikePrice'],axis=1)
dfsellput['Exit_Diff'] = dfsellput['Close_Buy'] - dfsellput['StrikePrice']
dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive if x['Pull_Up_Strike'] < 0 else profit_negative ,axis=1)
dfsellput['Profit'] = dfsellput.apply(lambda x: profit_positive // 2 if (x['Max_Days'] >= 8 and x['Profit'] < 0 and x['Pull_Up_Strike'] < 100) else x['Profit'], axis=1)
dfsellput['CumProfit'] = dfsellput['Profit'].cumsum()
dfsellput['CumProfitPercentage'] = dfsellput['CumProfit'] / 100000
dfsellput.head(5)

,Date_Buy,Open,High,Close_Buy,Gap,MonthYear,Date_Sell,Close_Sell,Year,BeginYear,...,Max_High,Max_Date,Gap_Open,Gap_Date,Max_Days,Pull_Up_Strike,Exit_Diff,Profit,CumProfit,CumProfitPercentage
1,2017-01-25,8499.45,8612.60,8602.75,23.65,20171,2017-01-13,8400.35,2017,0.0,...,8602.75,2017-01-25,8499.45,2017-01-25,12,-97.25,-97.25,2500,2500,0.025
2,2017-02-23,8956.40,8982.15,8939.50,105.70,20172,2017-02-10,8793.55,2017,0.0,...,8939.50,2017-02-23,8883.70,2017-02-17,13,-160.50,-160.50,2500,5000,0.050
3,2017-03-30,9142.60,9183.15,9173.75,54.10,20173,2017-03-15,9084.80,2017,0.0,...,9173.75,2017-03-30,9207.80,2017-03-17,15,-226.25,-226.25,2500,7500,0.075
4,2017-04-27,9359.15,9367.15,9342.15,55.10,20174,2017-04-17,9139.30,2017,0.0,...,9342.15,2017-04-27,9273.05,2017-04-25,10,-57.85,-57.85,2500,10000,0.100
5,2017-05-31,9636.55,9649.60,9621.25,52.35,20175,2017-05-16,9512.25,2017,0.0,...,9621.25,2017-05-31,9480.25,2017-05-22,15,-178.75,-178.75,2500,12500,0.125


In [15]:
dfsellput.drop(['Gap','Gap_Open','Gap_Date','Open','High','MonthYear','Year'],inplace=True,axis=1)
dfsellput.to_csv("nifty_sellcall_OTM" + str(OTM) +".csv")